In [1]:
import os
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import BingGroundingTool
from dotenv import load_dotenv

In [2]:
# Load environment variables from .env file
load_dotenv()

True

In [3]:
# Create an Azure AI Client from a connection string, copied from your Azure AI Foundry project.
# It should be in the format "<HostName>;<AzureSubscriptionId>;<ResourceGroup>;<HubName>"
# Customers need to login to Azure subscription via Azure CLI and set the environment variables

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=os.environ["PROJECT_CONNECTION_STRING"],
)

In [ ]:
# [START create_agent_with_bing_grounding_tool]
bing_connection = project_client.connections.get(connection_name=os.environ["BING_CONNECTION_NAME"])
conn_id = bing_connection.id

print(conn_id)

In [5]:
# Initialize agent bing tool and add the connection id
bing = BingGroundingTool(connection_id=conn_id)

In [6]:
# Create agent with the bing tool and process assistant run
with project_client:
    agent = project_client.agents.create_agent(
        model=os.environ["MODEL_DEPLOYMENT_NAME"],
        name="my-assistant",
        instructions="You are a helpful assistant",
        tools=bing.definitions,
        headers={"x-ms-enable-preview": "true"},
    )
    # [END create_agent_with_bing_grounding_tool]

    print(f"Created agent, ID: {agent.id}")

    # Create thread for communication
    thread = project_client.agents.create_thread()
    print(f"Created thread, ID: {thread.id}")

    # Create message to thread
    message = project_client.agents.create_message(
        thread_id=thread.id,
        role="user",
        content="What is the latest Microsoft stock price?",
    )
    print(f"Created message, ID: {message.id}")

    # Create and process agent run in thread with tools
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
        print(f"Run failed: {run.last_error}")

    # Delete the assistant when done
    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # Fetch and log all messages
    messages = project_client.agents.list_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

Created agent, ID: asst_s3lrnjP2tbJMQsCWCRkN8R9L
Created thread, ID: thread_XImCPca07aiIrIpyYpS4bKSN
Created message, ID: msg_gC5sUmDZHUBMPeTSpDNOBiKB
Run finished with status: RunStatus.COMPLETED
Deleted agent
Messages: {'object': 'list', 'data': [{'id': 'msg_4ObLLq2fHqC5nE11gTc3IF9Z', 'object': 'thread.message', 'created_at': 1740679576, 'assistant_id': 'asst_s3lrnjP2tbJMQsCWCRkN8R9L', 'thread_id': 'thread_XImCPca07aiIrIpyYpS4bKSN', 'run_id': 'run_zFBvlzh692XALcYrJiQQODjx', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'The latest stock price for Microsoft (MSFT) is $399.73 as of February 26, 2025【1†source】.', 'annotations': [{'type': 'url_citation', 'text': '【1†source】', 'start_index': 78, 'end_index': 88, 'url_citation': {'url': 'https://finance.yahoo.com/quote/MSFT/history/', 'title': 'Microsoft Corporation (MSFT) Stock Historical Prices & Data - Yahoo Finance'}}]}}], 'attachments': [], 'metadata': {}}, {'id': 'msg_gC5sUmDZHUBMPeTSpDNOBiKB', 'object': 'thread